In [1]:
from datasets import get_dataloaders

from easydict import EasyDict as edict
from train import train
import wandb

args = edict()
args.num_epochs = 50
args.pretrained_feats = False
args.pretrained_id = None #"6mwlbymd"
args.pretrained_epoch = 50
args.resume = True
args.experiment_id = "nef6r8hz" 
args.encoder = {
    'pretrained': False,
   # 'output_dim': 384, # vit_b_16: 784, vit_b_32: 784 # vit: 384.
    'id': args.pretrained_id,
    'epoch': args.pretrained_epoch,
    'arch': 'lvit',
    'frozen': False}

args.train_method = "encoder_erm"

args.seed = 444
args.test = True
args.save_weights = not args.test
args.save_metrics = not args.test
args.save_every = 10
args.num_workers = 0
METRICS_PER_METHOD = {
                      "erm": ['acc'],
                      "task_jepa": ['acc'], 
                      "ijepa": ['acc'],
                      "task_jepa+erm": ['acc'],
                      "pair_erm": ["acc"],
                      "encoder_erm": ['acc']
                     }

FOVS = {"shapes3d": {'floor_hue': 10, 'wall_hue': 10, 'object_hue': 10, 
                          'scale': 8, 'shape': 4, 'orientation': 15}}
FOVS_PER_DATASET = {'shapes3d': ["floor_hue", "wall_hue", "object_hue", "scale", "shape", "orientation"]}
#FOVS_PER_DATASET = {'shapes3d': ["floor_hue", "wall_hue", "scale", "shape", "orientation"]}

args.best_model_criterion = "val_avg_acc"
args.metrics = METRICS_PER_METHOD[args.train_method]
args.dataset = "shapes3d"
args.fovs =  FOVS_PER_DATASET[args.dataset]
args.fovs_tasks = ["floor_hue", "wall_hue", "object_hue", "scale", "shape", "orientation"]
args.fovs_indices = {name: i for i, name in enumerate(args.fovs)}
# When 2, 0 == same, 1 = different, When 3 levels, 0 = same, 1 = greater than, 2 = lower than
args.fovs_levels = {"floor_hue": 2, "wall_hue": 2, "object_hue": 2, "scale": 3, "shape": 2, "orientation": 3}
args.fovs_ids = [args.fovs_indices[x] for x in args.fovs_tasks ]
args.n_fovs = FOVS[args.dataset]
args.task_to_label_index = {k: i for i, k in enumerate(args.fovs_tasks)}
args.data_dir = "/mnt/nas2/GrimaRepo/araymond/3dshapes" 

# optimization
args.ema_start = 0.999 # starting value for exponential moving average for jepa methods
args.iters_per_ema = 16.0 # after how many iterations to apply ema
args.ipe_scale = 1.0
args.lr = 0.001
args.start_lr = 0.0002
args.final_lr = 1.0e-06
args.wd = 0.04
args.final_wd = 0.4
args.warmup = 2.0/15.0*args.num_epochs


/home/araymond/storage/pyenv/versions/3.10.14/envs/mini/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
dls = get_dataloaders(args)

In [3]:
import lightning as L
import torch
import torch.nn as nn
from models import VisionTransformer,LatentVisionTransformer, MultiHeadClassifier, PairVisionTransformer, MultiHeadClassifier
from lightning.pytorch.callbacks import ModelCheckpoint
from rich.table import Table
from rich.console import Console
from lightning.pytorch.loggers.wandb import WandbLogger
from lightning.pytorch.loggers.csv_logs import CSVLogger

class LightningEncoderERM(L.LightningModule):
    def __init__(self, args, encoder, predictor):
        super().__init__()
        self.encoder = encoder
        self.predictor = predictor
        self.criterion = nn.CrossEntropyLoss()
        self.args = args

    def forward(self, batch):
        x, y = batch
        bs, *_ = x[0].shape
        
        x = torch.cat([x[0] , x[1]], dim=0) # concatenate along batch dimension for efficiency
        features = encoder(x) 
        x_1 = features[:bs]
        x_2 = features[bs:]

        features = torch.cat([x_1, x_2], dim=-1)

        output = predictor(features)
        
        losses = []
        # decompose loss per task
        for i, fov in enumerate(args.fovs_tasks):
            losses.append(self.criterion(output[i], y[:,i]))
        
        return torch.stack(losses), [o.detach().cpu() for o in output]

    def training_step(self, batch, batch_idx):
        _, expected_output = batch
        expected_output = expected_output.cpu()
        losses, outputs = self(batch)
        loss = torch.mean(losses)
        metrics = {"train_loss": loss}
        
        for i, output in enumerate(outputs):
            fov = self.args.fovs_tasks[i]
            preds = output.argmax(dim=1)
            y = expected_output[:,i]
            correct = (preds == y).float().sum()
            total = y.numel()
            acc = correct/total
            metrics[f'train_{fov}_acc'] = acc
        self.log_dict(metrics, on_step=False, on_epoch=True, prog_bar=True)
        
        return loss

    def validation_step(self, batch, batch_idx, dataloader_idx=0):
        _, expected_output = batch
        expected_output = expected_output.cpu()
        losses, outputs = self(batch)
        loss = torch.mean(losses)
        split = ['val','test'][dataloader_idx]
        metrics = {f"{split}_loss": loss}
        
        for i, output in enumerate(outputs):
            fov = self.args.fovs_tasks[i]
            preds = output.argmax(dim=1)
            y = expected_output[:,i]
            correct = (preds == y).float().sum()
            total = y.numel()
            acc = correct/total
            metrics[f'{split}_{fov}_acc'] = acc
            
        self.log_dict(metrics, on_step=False, on_epoch=True, prog_bar=True, add_dataloader_idx=False)
        return loss


    def configure_optimizers(self):
        param_groups = [
                {'params': self.encoder.parameters()},
                {'params': self.predictor.parameters()}
            ]
        return torch.optim.AdamW(param_groups, lr=self.args.lr)





## Build Lightning Trainer

In [4]:
from models import create_model
from utils import get_exp_name


def create_loggers(args):
    loggers = []
    try:
        wandb_logger = WandbLogger(
            project='task_jepa',
            name="new_experiment",
            version=args.experiment_id,
        )
        wandb_logger.log_hyperparams(args)
        args.experiment_id = wandb_logger.version
        loggers.append(wandb_logger)
    except wandb.errors.CommError as e:
        print(e)
        pass
    
    output_path = f"results/{args.experiment_id}"
    csv_logger = CSVLogger(
        output_path,
        version = "v1",
        name=''
    )
    csv_logger.log_hyperparams(args)
    loggers.append(csv_logger)
    
    return loggers
    
def create_callbacks(args):
    callbacks = []
    save_top_k=-1
    dir_path = f"results/{args.experiment_id}"
    callbacks.append(ModelCheckpoint(dirpath=dir_path,
                                 filename="{epoch}",
                                 save_top_k=save_top_k,
                                 monitor="val_loss",
                                 every_n_epochs=5,
                                 save_last=True))
    return callbacks


loggers = create_loggers(args)
args.experiment_name = get_exp_name(args)
callbacks = create_callbacks(args)
wandb.run.name = args.experiment_name


encoder = LatentVisionTransformer(
    image_size=64,
    patch_size=8,
    num_layers=4,
    num_heads=12,
    hidden_dim=384,
    mlp_dim=128,
    num_classes=1
    )
output_dims = [args.fovs_levels[k] for k in args.fovs_tasks]
encoder.heads = nn.Identity()
predictor = MultiHeadClassifier(input_dim=2*384, output_dims=output_dims)
#encoder, predictor = create_model(args)

model = LightningEncoderERM(args, encoder,predictor)

ckpt_path = f"results/{args.experiment_id}/last.ckpt" if args.resume else None

trainer = L.Trainer(accelerator="gpu",
                    devices=1,
                    max_epochs=args.num_epochs,
                    callbacks=callbacks,
                    logger=loggers)
trainer.fit(model=model,
            train_dataloaders=dls['train'],
            val_dataloaders=[dls['val'],dls['test']],
            ckpt_path = ckpt_path)
wandb.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alainray (alainray_puc). Use `wandb login --relogin` to force relogin


/home/araymond/storage/pyenv/versions/3.10.14/envs/mini/lib/python3.10/site-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory results/nef6r8hz/v1 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/araymond/storage/pyenv/versions/3.10.14/envs/mini/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:653: Checkpoint directory /mnt/ialabnas/homes/araymond/jepa/results/nef6r8hz exists and is not empty.
Restoring states from the checkpoint path at results/nef6r8hz/last.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [5]

  | Name      | Type                    | Params
------------------------------------------------------
0 | encoder   | LatentVisionTransformer | 2.9 M 
1 | predictor | MultiHeadClassifier     | 100 K 
2 | criterion | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/araymond/storage/pyenv/versions/3.10.14/envs/mini/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
test_floor_hue_acc,▂▁▁▁▁▁▁▁▂▂▂▅▅▄▄▇▅▅▄▄██▇▅▅▅▅▅
test_loss,▃▃▁▁▁▁▁▁▁▁▂▂▂▃▃▄▄▅▆▆▆▆▆▆▇▇▇█
test_object_hue_acc,▁▅▆▇▆▇▆▇▇▇▇▇▇▆▇▇██▇▇▆▅▆▆▆█▅▄
test_orientation_acc,▁▁▁▂▃▃▄▅▅▅▅▅▅▆▇▇▇▇▇▇▇███████
test_scale_acc,▇███▇▇▇█▇▇▆▆▅▄▄▄▂▂▂▁▁▁▁▁▁▁▂▂
test_shape_acc,▂▅▇▄▂▁▄▃▄▅▅▅▅▅▅▆▅▇█▆█▆▆▆▇▅▅▅
test_wall_hue_acc,▄▂▁▁▁▂▂▂▂▂▄▄▂▂▄▅▅▇▇▇▇████▅▅▄
train_floor_hue_acc,▂▁▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▅▇▇▆▇█
train_loss,█▆▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train_object_hue_acc,▁▄▄▅▄▅▅▅▅▅▄▅▅▄▄▅▅▅▅▅▆▇▇▇▇▇██


In [6]:
wandb.finish()

In [4]:
train(args, dls)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alainray (alainray_puc). Use `wandb login --relogin` to force relogin


Experiment id is 7np74doo
Encoder is unfrozen!


  0%|          | 0/1000 [00:00<?, ?it/s]

EPOCH 1


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Iteration: 1
Floor_hue:
  - Acc: Avg = 0.8867
  - Loss: Avg = 0.3289

Wall_hue:
  - Acc: Avg = 0.8867
  - Loss: Avg = 0.3901

Object_hue:
  - Acc: Avg = 0.9023
  - Loss: Avg = 0.3940

Scale:
  - Acc: Avg = 0.5781
  - Loss: Avg = 2.0887

Shape:
  - Acc: Avg = 0.5312
  - Loss: Avg = 1.2728

Orientation:
  - Acc: Avg = 0.6602
  - Loss: Avg = 2.0243

[VAL]
Floor_hue:
  - Acc: Avg = 0.8867
  - Loss: Avg = 0.3289

Wall_hue:
  - Acc: Avg = 0.8867
  - Loss: Avg = 0.3901

Object_hue:
  - Acc: Avg = 0.9023
  - Loss: Avg = 0.3940

Scale:
  - Acc: Avg = 0.5781
  - Loss: Avg = 2.0887

Shape:
  - Acc: Avg = 0.5312
  - Loss: Avg = 1.2728

Orientation:
  - Acc: Avg = 0.6602
  - Loss: Avg = 2.0243



  0%|          | 0/1 [00:00<?, ?it/s]

Iteration: 1
Floor_hue:
  - Acc: Avg = 0.8281
  - Loss: Avg = 0.3790

Wall_hue:
  - Acc: Avg = 0.8477
  - Loss: Avg = 0.3568

Object_hue:
  - Acc: Avg = 0.8125
  - Loss: Avg = 0.4875

Scale:
  - Acc: Avg = 0.3438
  - Loss: Avg = 3.7109

Shape:
  - Acc: Avg = 0.5117
  - Loss: Avg = 1.9030

Orientation:
  - Acc: Avg = 0.3516
  - Loss: Avg = 5.2490

[TEST]
Floor_hue:
  - Acc: Avg = 0.8281
  - Loss: Avg = 0.3790

Wall_hue:
  - Acc: Avg = 0.8477
  - Loss: Avg = 0.3568

Object_hue:
  - Acc: Avg = 0.8125
  - Loss: Avg = 0.4875

Scale:
  - Acc: Avg = 0.3438
  - Loss: Avg = 3.7109

Shape:
  - Acc: Avg = 0.5117
  - Loss: Avg = 1.9030

Orientation:
  - Acc: Avg = 0.3516
  - Loss: Avg = 5.2490

Classifier 1 classifiers.0.weight Parameter containing:
tensor([[-0.0332,  0.0095,  0.0102,  ...,  0.0354, -0.0020,  0.0077],
        [-0.0208,  0.0370, -0.0098,  ..., -0.0581,  0.0389,  0.0190],
        [ 0.0123, -0.0058, -0.0202,  ...,  0.0409, -0.0210,  0.0206],
        ...,
        [ 0.0003,  0.0266, -0

KeyboardInterrupt: 